In [1]:
import random

known_weather_data = {"berlin": 20.0}


def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the weather of a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "a city in the mentioned country or mentioned city name",
            }
        },
        "required": ["city"],
        "additionalProperties": False,
    },
}

In [3]:
import json
from openai import OpenAI
from IPython.display import display, HTML
import markdown


class Tools:
    def __init__(self):
        self.tools = {}
        self.functions = {}

    def add_tool(self, function, description):
        self.tools[function.__name__] = description
        self.functions[function.__name__] = function

    def get_tools(self):
        return list(self.tools.values())

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        f = self.functions[function_name]
        result = f(**arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }


def shorten(text, max_length=50):
    if len(text) <= max_length:
        return text

    return text[: max_length - 3] + "..."


class ChatInterface:
    def input(self):
        question = input("You:")
        return question

    def display(self, message):
        print(message)

    def display_function_call(self, entry, result):
        call_html = f"""
            <details>
            <summary>Function call: <tt>{entry.name}({shorten(entry.arguments)})</tt></summary>
            <div>
                <b>Call</b>
                <pre>{entry}</pre>
            </div>
            <div>
                <b>Output</b>
                <pre>{result["output"]}</pre>
            </div>
            
            </details>
        """
        display(HTML(call_html))

    def display_response(self, entry):
        response_html = markdown.markdown(entry.content[0].text)
        html = f"""
            <div>
                <div><b>Assistant:</b></div>
                <div>{response_html}</div>
            </div>
        """
        display(HTML(html))


class ChatAssistant:
    def __init__(self, tools, developer_prompt, chat_interface, client):
        self.tools = tools
        self.developer_prompt = developer_prompt
        self.chat_interface = chat_interface
        self.client = client

    def gpt(self, chat_messages):
        return self.client.responses.create(
            model="gpt-4o-mini",
            input=chat_messages,
            tools=self.tools.get_tools(),
        )

    def run(self):
        chat_messages = [
            {"role": "developer", "content": self.developer_prompt},
        ]

        # Chat loop
        while True:
            question = self.chat_interface.input()
            if question.strip().lower() == "stop":
                self.chat_interface.display("Chat ended.")
                break

            message = {"role": "user", "content": question}
            chat_messages.append(message)

            while True:  # inner request loop
                response = self.gpt(chat_messages)

                has_messages = False

                for entry in response.output:
                    chat_messages.append(entry)

                    if entry.type == "function_call":
                        result = self.tools.function_call(entry)
                        chat_messages.append(result)
                        self.chat_interface.display_function_call(entry, result)

                    elif entry.type == "message":
                        self.chat_interface.display_response(entry)
                        has_messages = True

                if has_messages:
                    break


In [7]:
tools = Tools()
tools.add_tool(get_weather, get_weather_tool)

chat_interface = ChatInterface()
client = OpenAI()

In [10]:
chat_assistant = ChatAssistant(
    tools,
    "You are a helpful assistant that can get the weather of a city. If the user asks for the weather of a country, you should return the weather of the capital of the country.",
    chat_interface,
    client,
)

In [11]:
chat_assistant.run()

Chat ended.


In [4]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Add weather data to our database",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "a city in the mentioned country or mentioned city name",
            },
            "temp": {
                "type": "number",
                "description": "the temperature of the city",
            },
        },
        "required": ["city", "temp"],
        "additionalProperties": False,
    },
}

In [5]:
tools = Tools()
tools.add_tool(set_weather, set_weather_tool)
tools.add_tool(get_weather, get_weather_tool)

In [6]:
tools.get_tools()

[{'type': 'function',
  'name': 'set_weather',
  'description': 'Add weather data to our database',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'a city in the mentioned country or mentioned city name'},
    'temp': {'type': 'number', 'description': 'the temperature of the city'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'get_weather',
  'description': 'Get the weather of a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'a city in the mentioned country or mentioned city name'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [7]:
chat_interface = ChatInterface()
client = OpenAI()

chat_assistant = ChatAssistant(
    tools,
    "You are a helpful assistant that can get the weather of a city and set the weather of a city. If the user asks for the weather of a country, you should return the weather of the capital of the country.  ",
    chat_interface,
    client,
)

In [8]:
chat_assistant.run()

Chat ended.
